In [1]:
import numpy as np

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
class LogicGate:
        
    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        
        # 2층 hidden layer unit : 6개 가정,  가중치 W2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2,20)  # weight, 2 X 6 matrix
        self.__b2 = np.random.rand(20)
        
        # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(20,1)
        self.__b3 = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
    
        print(self.name + " object is created")
            
    def feed_forward(self):        # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )    
    
    def loss_val(self):          # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )
    
    
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in  range(10001):
            
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
    
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
        
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
    
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
    
            if (step % 400 == 0):
                print("\n*****************************************************************************")
                print("step = ", step, "  , loss value = ", self.loss_val())
                print("\nW2 = " , self.__W2, "\n")
                print("b2 = " , self.__b2, "\n")
                
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

In [3]:
# AND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)

and_obj.train()

AND object is created
Initial loss value =  21.461894455975496

*****************************************************************************
step =  0   , loss value =  20.56526521668512

W2 =  [[0.48017266 0.88879393 0.84835772 0.23245704 0.09159339 0.59320042
  0.00774443 0.69269256 0.31949323 0.70694777 0.33412067 0.21847837
  0.08068826 0.1692875  0.50825522 0.9884152  0.1477579  0.19922495
  0.09670485 0.22887556]
 [0.94508354 0.68132428 0.85571899 0.90014821 0.23656594 0.80311338
  0.193813   0.30124251 0.13032968 0.47918216 0.06664714 0.19116183
  0.01120265 0.85747666 0.64559118 0.5128851  0.35122245 0.12080022
  0.08414026 0.83419475]] 

b2 =  [0.62130937 0.32656896 0.29122758 0.12733892 0.33674828 0.61257237
 0.20366154 0.02096092 0.06751012 0.56781946 0.47471076 0.42989503
 0.21439235 0.11070922 0.84207389 0.39117571 0.32827041 0.74736265
 0.73632025 0.06958151] 


*****************************************************************************
step =  400   , loss value =  2.


*****************************************************************************
step =  4000   , loss value =  0.18510291100839094

W2 =  [[ 0.68794054  0.75869676  1.52007127  0.83773572 -0.64715247  0.59624986
  -0.11947608  1.26770692 -0.769947    0.87439636  0.08911215 -0.18543869
  -0.33278333  0.06304377  0.60106503  1.3668637  -1.32881179 -1.04552836
  -1.46219274  0.47532632]
 [ 1.31683219  0.60502737  1.41260546  1.61571199 -0.46861121  0.81240656
   0.07351554  0.72598149 -0.89825253  0.57405946 -0.15130592 -0.18992202
  -0.37778149  0.73349954  0.74724463  0.64409746 -1.03700491 -1.04207843
  -1.36866609  1.14419103]] 

b2 =  [-0.58877302  0.82642333 -1.73360963 -1.19068958  0.48043208  0.50378337
  0.13673527 -0.76192544  0.60963182  0.14718879  0.46404314  0.46108716
  0.22285275  0.21299352  0.45234072 -0.6049255   1.23909866  1.12580039
  1.67757131 -0.33854572] 


*****************************************************************************
step =  4400   , loss value = 


*****************************************************************************
step =  8000   , loss value =  0.05188182429285449

W2 =  [[ 0.76672088  0.75293873  1.76051697  1.0181705  -0.84915549  0.60216427
  -0.20071092  1.40391474 -0.95949229  0.91801148 -0.01515911 -0.34154623
  -0.49186937  0.0767427   0.61610744  1.48508144 -1.5751331  -1.29197379
  -1.7515525   0.54183381]
 [ 1.4289873   0.60269897  1.65847477  1.77895861 -0.66334284  0.82348025
  -0.00737768  0.84027536 -1.08779516  0.59923764 -0.24905305 -0.34393344
  -0.5351907   0.75208779  0.76740421  0.71733856 -1.29042443 -1.2822602
  -1.66221569  1.24214665]] 

b2 =  [-0.86368538  0.8540768  -2.17815712 -1.5439075   0.61167183  0.46307132
  0.15051494 -1.01648729  0.90266073  0.04273105  0.47796342  0.48547023
  0.29202061  0.202869    0.38863394 -0.86664639  1.67990966  1.45608964
  2.17830239 -0.49188175] 


*****************************************************************************
step =  8400   , loss value =  

In [4]:
# AND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(and_obj.predict(data))

(array([4.31949344e-06]), 0)
(array([0.00931594]), 0)
(array([0.0092088]), 0)
(array([0.98274229]), 1)


In [5]:
# OR Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)

or_obj.train()

OR object is created
Initial loss value =  6.960682899437499

*****************************************************************************
step =  0   , loss value =  6.8647837582515905

W2 =  [[0.93559587 0.5742627  0.71984849 0.91160845 0.1424119  0.05939683
  0.66385587 0.0627749  0.3477063  0.61612625 0.89954615 0.55157481
  0.41706776 0.47018125 0.53346006 0.33387292 0.67567821 0.50427808
  0.81503794 0.98604818]
 [0.51026794 0.94130564 0.48439555 0.88906393 0.62326733 0.54528238
  0.62087788 0.56292904 0.54793907 0.13625294 0.69680534 0.45873498
  0.5217953  0.87120046 0.18929151 0.90705208 0.90170334 0.72269307
  0.47700337 0.96123513]] 

b2 =  [0.97571322 0.41381734 0.51192138 0.60769334 0.70529211 0.59524004
 0.82641211 0.53761593 0.71678838 0.23069508 0.43359883 0.46670843
 0.30279641 0.84074828 0.94369333 0.3220503  0.24429778 0.49432462
 0.89371525 0.68804531] 


*****************************************************************************
step =  400   , loss value =  1.8


*****************************************************************************
step =  3600   , loss value =  0.1534505785815546

W2 =  [[ 8.25588688e-01  3.85078815e-01  1.87665568e+00  1.52186125e+00
  -3.27109384e-01 -2.63069038e-01  1.51309960e+00  7.79985084e-01
   9.60919992e-01  1.65998436e+00  1.48127324e+00 -7.09919725e-02
  -3.44381306e-01 -4.38145309e-04  2.89891558e-01  4.91770486e-01
   1.38288143e+00  8.73875712e-01  3.56193941e-01  2.08421499e+00]
 [ 3.60238992e-01  7.73560360e-01  1.77021939e+00  1.53055084e+00
   2.16852652e-01  2.56763012e-01  1.51584895e+00  1.21279491e+00
   1.14217031e+00  1.34215953e+00  1.35006377e+00 -2.14122312e-01
  -2.52941423e-01  4.83408788e-01 -1.14214594e-01  1.04172304e+00
   1.58474630e+00  1.07808337e+00 -1.15735733e-01  2.11147506e+00]] 

b2 =  [ 1.16876627  0.66539353 -0.83029233 -0.44013033  0.95315718  0.73214278
 -0.4884478  -0.1592171  -0.05928115 -0.67519421 -0.41246916  0.87063637
  0.69614509  1.38074137  1.0977161   0.1541685


*****************************************************************************
step =  7600   , loss value =  0.039980881606692616

W2 =  [[ 0.788313    0.3539721   2.190981    1.7412063  -0.50018129 -0.40603773
   1.75698003  0.9340813   1.10222361  1.91486754  1.68239414 -0.25776033
  -0.57418673 -0.1264327   0.18435584  0.54952184  1.62647819  0.98151442
   0.24251092  2.43330708]
 [ 0.30694717  0.74768488  2.1084241   1.7549058   0.07353917  0.1328335
   1.76597778  1.34714014  1.2763457   1.63809864  1.56884987 -0.41524476
  -0.4860478   0.38977373 -0.24833733  1.08896082  1.81465893  1.17916369
  -0.27607021  2.46496584]] 

b2 =  [ 1.23733072  0.69939928 -1.03675803 -0.67173635  1.00984522  0.79091661
 -0.71949066 -0.29382158 -0.21419178 -0.84443857 -0.616166    0.9214869
  0.74909634  1.46611561  1.15844853  0.09854245 -0.73900075 -0.09561928
  1.5583141  -1.19492444] 


*****************************************************************************
step =  8000   , loss value =  

In [6]:
# OR Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(or_obj.predict(data))

(array([0.01436189]), 0)
(array([0.99422894]), 1)
(array([0.99432378]), 1)
(array([0.99994309]), 1)


In [7]:
# NAND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

NAND object is created
Initial loss value =  9.85939876645908

*****************************************************************************
step =  0   , loss value =  9.713068464257534

W2 =  [[ 0.49168705  0.93903426  0.28177284  0.30037254  0.23208069  0.20541093
   0.81287259  0.05944035  0.00762649  0.09847521  0.9729015   0.42327124
   0.47558744  0.43870746  0.52217203  0.28116782  0.92384968  0.84623142
   0.42775993  0.24679522]
 [ 0.25042508  0.12322916 -0.0014066   0.6587221   0.01244101  0.51509312
   0.42937759  0.99347403  0.8285409   0.78545644  0.7501778   0.32104582
  -0.001229    0.25329249  0.87137554  0.06921704  0.71044889  0.14718587
   0.06508705  0.80295855]] 

b2 =  [0.82697953 0.16672382 0.45812253 0.84652674 0.80396275 0.54032375
 0.69627345 0.7606198  0.15841851 0.65890699 0.53410069 0.79875754
 0.71050227 0.88016726 0.88298574 0.8705949  0.22872543 0.57394989
 0.25234437 0.21836616] 


***********************************************************************


*****************************************************************************
step =  4000   , loss value =  0.16772301569002424

W2 =  [[-0.9868352   0.40519373 -1.18318926  0.43003654 -0.82593665 -1.10433461
   1.28653235 -0.08258123  0.66800027  0.42848054  1.14562327  0.66840479
  -1.43669375  0.5646409   0.63327053 -1.31468893  1.46661841  0.30028485
  -0.7909976   0.17663329]
 [-0.98068007 -0.29003213 -1.17970555  0.79371755 -0.8765754  -0.64472094
   0.51778397  0.80640479  1.52981159  1.23768459  0.72863129  0.4755988
  -1.4914316   0.33086552  0.99512403 -1.23810298  0.9095784  -0.21913154
  -0.95633631  0.74186061]] 

b2 =  [ 1.07471539  0.21533398  1.18290406  0.58648802  0.93112413  0.79550529
 -0.22625961  0.85122086 -0.8283274  -0.06465301 -0.16231644  0.51974737
  1.7096421   0.75281234  0.30698561  1.40739164 -1.04656552  0.7167784
  0.64354524  0.17119505] 


*****************************************************************************
step =  4400   , loss value =  0


*****************************************************************************
step =  8000   , loss value =  0.04990342005088592

W2 =  [[-1.22302659  0.35742213 -1.40392099  0.44691529 -1.0494333  -1.3165427
   1.39767752 -0.08567081  0.78809265  0.46167459  1.2076793   0.70713918
  -1.71569657  0.58659378  0.64337622 -1.55277681  1.62318197  0.21993691
  -0.96536569  0.21322767]
 [-1.21359757 -0.33342644 -1.3962272   0.82604268 -1.09679677 -0.84582822
   0.558902    0.79903402  1.68199164  1.33810779  0.75369882  0.50445905
  -1.75893787  0.34718986  1.03206779 -1.47641819  1.05612264 -0.28516872
  -1.13457785  0.79026116]] 

b2 =  [ 1.3123977   0.21690715  1.56407053  0.52899916  1.1022372   1.01195093
 -0.4447754   0.85536685 -1.11419249 -0.24317757 -0.36329217  0.45877777
  2.17834105  0.72978997  0.17654373  1.78803398 -1.37785206  0.72168474
  0.89439705  0.14301188] 


*****************************************************************************
step =  8400   , loss value =  

In [8]:
# NAND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(nand_obj.predict(data))

(array([0.99999769]), 1)
(array([0.99115315]), 1)
(array([0.9911965]), 1)
(array([0.01720539]), 0)


In [9]:
# XOR Gate 객체 생성

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XOR object is created
Initial loss value =  16.56004496240316

*****************************************************************************
step =  0   , loss value =  16.096337316021163

W2 =  [[0.86472321 0.03110738 0.62704581 0.14756317 0.40562026 0.86196552
  0.90003297 0.72755986 0.77676461 0.95841756 0.35187183 0.14872642
  0.63339978 0.9682456  0.25243257 0.81062909 0.94917526 0.57071326
  0.05959387 0.57779978]
 [0.6945205  0.86389943 0.426144   0.71801078 0.76601742 0.79751867
  0.92458945 0.82736917 0.14344613 0.37127477 0.28884693 0.45759071
  0.17339833 0.64823722 0.78431784 0.39817633 0.50047641 0.84214678
  0.73032288 0.55123865]] 

b2 =  [0.56595468 0.23748151 0.55245651 0.83054746 0.11347698 0.02784497
 0.07527186 0.25006523 0.88275446 0.61767081 0.26478762 0.45608683
 0.18127783 0.60212693 0.79939472 0.1094074  0.76853781 0.21311021
 0.42698243 0.66605733] 


*****************************************************************************
step =  400   , loss value =  2.


*****************************************************************************
step =  4000   , loss value =  2.096587757173559

W2 =  [[ 0.73423342  0.1340009   0.65845587 -0.31765444  0.2998651   2.25819762
   2.22994396  0.54751293  1.2916584   1.37721707  0.41273806  0.15362022
   0.92045424  1.7683607  -0.43132462  0.82544832  0.89109007  0.43867155
   0.13711679  0.48212055]
 [ 0.5118817   0.73854358  0.47853584  1.06048057  0.79412528  2.24281952
   2.23626281  0.70004695 -0.54880071  1.1400057   0.33525461  0.39896512
   0.04892718  1.6663404   1.22588054  0.30082224  0.19357974  0.78781922
   0.65769263  0.45311211]] 

b2 =  [ 0.49966594  0.14703872  0.52063437  0.75592035 -0.05928793  0.108816
  0.11685761  0.0982128   0.82260416  0.6797036   0.16823853  0.37719459
  0.08017945  0.52826831  0.73083106 -0.04302356  0.69820832  0.07796639
  0.34311943  0.58146224] 


*****************************************************************************
step =  4400   , loss value =  1.9


*****************************************************************************
step =  8000   , loss value =  0.497658863442615

W2 =  [[ 7.88371086e-01  2.10573961e-01  6.81476363e-01 -1.02422906e+00
   5.78005569e-01  3.78152766e+00  3.72942006e+00  6.77817601e-01
   3.53261403e+00  1.90707973e+00  7.37310108e-01  2.24684824e-01
   1.85001192e+00  2.74728852e+00 -1.54045928e+00  1.28339333e+00
   1.16592560e+00  5.40737699e-01  1.88642665e-01  5.68880771e-01]
 [ 4.65771127e-01  9.34008866e-01  4.71397252e-01  2.46317749e+00
   1.26114326e+00  3.78511209e+00  3.73792543e+00  8.89219727e-01
  -1.89878567e+00  1.80066447e+00  6.05198792e-01  4.94157464e-01
   1.26285021e-01  2.71863061e+00  3.17897717e+00  5.07078587e-01
   2.97353096e-03  1.04079676e+00  8.20253478e-01  5.16875003e-01]] 

b2 =  [ 0.20518154 -0.031718    0.40928156  0.44491755 -0.86318543 -0.6047097
 -0.56726742 -0.54076598  0.72516455  0.92736309 -0.31864456  0.31365927
 -0.69489425  0.29303822  0.56531629 -0.78441921 

In [10]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.02667246]), 0)
(array([0.93578725]), 1)
(array([0.93798919]), 1)
(array([0.0905949]), 0)
